In [1]:
import os
import shutil
from glob import glob
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv('./data/aug_train.csv')

# label 전처리

In [3]:
label_df = pd.DataFrame(columns=['cat1','cat2','cat3','count'])
for n_cat1 in df['cat1'].unique() :
    cat2 = df[df['cat1'] == n_cat1]['cat2'].unique()
    for n_cat2 in cat2 :
        cat3  = df[df['cat1'] == n_cat1][df['cat2'] == n_cat2]['cat3'].unique()
        for n_cat3 in cat3 :
            cnt = len(df[df['cat1'] == n_cat1][df['cat2'] == n_cat2][df['cat3'] == n_cat3])
            n = pd.DataFrame([[n_cat1, n_cat2, n_cat3, cnt]], columns=['cat1', 'cat2', 'cat3', 'count'])
            label_df = pd.concat([label_df, n], ignore_index=True)

label_df

,cat1,cat2,cat3,count
0,자연,자연관광지,항구/포구,134
1,자연,자연관광지,섬,111
2,자연,자연관광지,자연휴양림,118
3,자연,자연관광지,해수욕장,207
4,자연,자연관광지,산,239
...,...,...,...,...
123,쇼핑,쇼핑,상설시장,270
124,쇼핑,쇼핑,"공예,공방",41
125,쇼핑,쇼핑,백화점,16
126,쇼핑,쇼핑,면세점,18


In [4]:
cat_encoder = {v : i for i, v in enumerate(label_df.loc[:, 'cat3'].values)}
cat_encoder

{'항구/포구': 0,
 '섬': 1,
 '자연휴양림': 2,
 '해수욕장': 3,
 '산': 4,
 '수목원': 5,
 '강': 6,
 '자연생태관광지': 7,
 '계곡': 8,
 '폭포': 9,
 '국립공원': 10,
 '해안절경': 11,
 '호수': 12,
 '동굴': 13,
 '도립공원': 14,
 '군립공원': 15,
 '약수터': 16,
 '등대': 17,
 '희귀동.식물': 18,
 '기암괴석': 19,
 '골프': 20,
 '야영장,오토캠핑장': 21,
 '스키(보드) 렌탈샵': 22,
 '자동차경주': 23,
 '자전거하이킹': 24,
 '썰매장': 25,
 '승마': 26,
 '트래킹': 27,
 '수련시설': 28,
 '카지노': 29,
 '번지점프': 30,
 '카트': 31,
 'MTB': 32,
 '스케이트': 33,
 '인라인(실내 인라인 포함)': 34,
 '사격장': 35,
 'ATV': 36,
 '빙벽등반': 37,
 '스키/스노보드': 38,
 '복합 레포츠': 39,
 '요트': 40,
 '래프팅': 41,
 '민물낚시': 42,
 '바다낚시': 43,
 '수영': 44,
 '카약/카누': 45,
 '윈드서핑/제트스키': 46,
 '스노쿨링/스킨스쿠버다이빙': 47,
 '스카이다이빙': 48,
 '헹글라이딩/패러글라이딩': 49,
 '수상레포츠': 50,
 '한식': 51,
 '일식': 52,
 '바/까페': 53,
 '채식전문점': 54,
 '중식': 55,
 '서양식': 56,
 '패밀리레스토랑': 57,
 '클럽': 58,
 '일반축제': 59,
 '문화관광축제': 60,
 '유적지/사적지': 61,
 '성': 62,
 '안보관광': 63,
 '사찰': 64,
 '종교성지': 65,
 '고택': 66,
 '고궁': 67,
 '민속마을': 68,
 '문': 69,
 '생가': 70,
 '전시관': 71,
 '미술관/화랑': 72,
 '박물관': 73,
 '도서관': 74,
 '공연장': 75,
 '기념관': 76,
 '

# FSL에 학습 규격에 맞춰서 이미지 이동

### 경로 만들기

In [84]:
root_path = './data/fsl_img'
os.makedirs(root_path, exist_ok=True)

for i, label_name in enumerate(label_df['cat3'].values) :
    fold_path = os.path.join(root_path, str(i))#label_name)
    os.makedirs(fold_path, exist_ok=True)

### 이미지 이동

In [85]:
# df.iloc[0]
source_root_path = './data'
dest_root_path = './data/fsl_img'

for i in tqdm(range(len(df))):
    img_path = os.path.join(source_root_path, df.iloc[i]['img_path'])
    label = cat_encoder[df.iloc[i]['cat3']] #df.iloc[i]['cat3'] 
    img_name = df.iloc[i]['id']
    dest_path = os.path.join(dest_root_path, str(label), img_name+'.jpg')
    
    # copy
    shutil.copyfile(img_path, dest_path)
    

100%|███████████████████████████████████████████████████████████████████████████| 17345/17345 [00:18<00:00, 955.26it/s]


# FSL에 이용하는 json 파일 생성

In [21]:
import random
from collections import defaultdict
import json

fsl_t = defaultdict(list)
fsl_v = defaultdict(list)

vaild_pick = random.sample([i for i in range(128)], 26)

path = './data/fsl_img/*'
label_list = glob(path)
label_list.sort(key=lambda x: int(x.split('\\')[-1]))



for idx, label in enumerate(label_list) :  
    if idx in vaild_pick :
        fsl_v['class_roots'].append(label)
        fsl_v['class_names'].append(label.split('\\')[-1])

    else :
        fsl_t['class_roots'].append(label)
        fsl_t['class_names'].append(label.split('\\')[-1])

    
with open('./data/fsl_train.json', 'w') as f:
    json.dump(fsl_t, f, indent=2)
    
with open('./data/fsl_valid.json', 'w') as f:
    json.dump(fsl_v, f, indent=2)


# FSL configuration 설정

In [5]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import timm
import os

from easyfsl.methods.utils import evaluate
from easydict import EasyDict

args = EasyDict({'n_way' : 4, 
                 'n_shot' : 3, 
                 'n_query' : 6, 
                 'n_workers' : 6, 
                 'DEVICE' : "cuda"})

# DataLoader

In [6]:
from easyfsl.datasets import CUB, CUSTOM
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


n_tasks_per_epoch = 300
n_validation_tasks = 100



train_set = CUSTOM(split="data/fsl_train", image_size=224 ,training=True)
val_set = CUSTOM(split="data/fsl_valid", image_size=224, training=False)

train_sampler = TaskSampler(
    train_set, 
    n_way=args.n_way, 
    n_shot=args.n_shot, 
    n_query=args.n_query, 
    n_tasks=n_tasks_per_epoch
)
val_sampler = TaskSampler(
    val_set, 
    n_way=args.n_way, 
    n_shot=args.n_shot, 
    n_query=args.n_query, 
    n_tasks=n_validation_tasks
)

train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=args.n_workers,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=args.n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

# Model

In [7]:
class CNN(nn.Module) :
    def __init__(self, name='efficientnetv2_rw_m', num_classes=0)  :
        super(CNN, self).__init__()
        self.model = timm.create_model(name, pretrained=True, num_classes=num_classes, global_pool='')
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten(1)
        
    def forward(self, x) :
        x = self.model.forward_features(x)
        x = self.pool(x)
        x = self.flatten(x)
        return x

In [8]:
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier, TIM, RelationNetworks

convolutional_network = CNN().to(args.DEVICE)
few_shot_classifier = PrototypicalNetworks(convolutional_network).to(args.DEVICE)


# Episodic Train

In [10]:
from torch.optim import SGD, Optimizer, AdamW, Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


criterion = nn.CrossEntropyLoss()

n_epochs = 30
scheduler_milestones = [3, 6]
scheduler_gamma = 0.1
learning_rate = 1e-3


train_optimizer = Adam(
    few_shot_classifier.parameters(), lr=learning_rate, weight_decay=5e-4)#,  momentum=0.9)


train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)


In [11]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.to(args.DEVICE), support_labels.to(args.DEVICE)
            )
            classification_scores = model(query_images.to(args.DEVICE))

            loss = criterion(classification_scores, query_labels.to(args.DEVICE))
#             loss.requires_grad_(True)
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [12]:
from easyfsl.methods.utils import evaluate

start_e = 0
best_e = None
resum = False
model_save_path = './ckpt/fsl/albumen_224_effi2_m_2'
os.makedirs(model_save_path, exist_ok=True)

tb_logs_dir = Path(".")
tb_writer = SummaryWriter(log_dir='./tensorboard/fsl_albumen_224_effi2_m_2')


best_state = few_shot_classifier.state_dict()
if resum == True : 
    fsl_checkpoint = torch.load(os.paht.join(model_save_path,'2E_model.pt'))
    few_shot_classifier.load_state_dict(fsl_checkpoint["model_state_dict"])
    train_optimizer.load_state_dict(fsl_checkpoint['optimizer_state_dict'])
    start_e = fsl_checkpoint["epoch"]
    
best_validation_accuracy = 0.0
for epoch in range(start_e, n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=args.DEVICE, tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        print("Ding ding ding! We found a new best model!")
        best_e = epoch
        torch.save({
                "epoch" : epoch,
                "model_state_dict" : best_state,
                "optimizer_state_dict" : train_optimizer.state_dict()
            }, os.path.join(model_save_path, str(epoch)+'E_model.pt'))
    

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Validation: 100%|████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.79it/s, accuracy=0.549]


Ding ding ding! We found a new best model!
Epoch 1


Validation: 100%|████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.87it/s, accuracy=0.486]


Epoch 2


Training:  15%|█████████                                                   | 45/300 [00:28<02:40,  1.59it/s, loss=0.88]


KeyboardInterrupt: 

# Classical Train

# Configuration

In [5]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import timm
import os

from easyfsl.methods.utils import evaluate
from easydict import EasyDict

args = EasyDict({'n_way' : 4, 
                 'n_shot' : 3, 
                 'n_query' : 6, 
                 'n_workers' : 6, 
                 'batch_size' : 32,
                 'n_validation_tasks' :200,
                 'DEVICE' : "cuda"})

# DataLoader

In [6]:
from easyfsl.datasets import CUSTOM
from easyfsl.samplers import TaskSampler
from torch.utils.data import DataLoader


train_set = CUSTOM(split="data/fsl_train", image_size=224, training=True)
train_loader = DataLoader(
    train_set,
    batch_size=args.batch_size,
    num_workers=args.n_workers,
    pin_memory=True,
    shuffle=True,
)


val_set = CUSTOM(split="data/fsl_valid", image_size=224, training=True)

val_sampler = TaskSampler(
    val_set, 
    n_way=args.n_way, 
    n_shot=args.n_shot, 
    n_query=args.n_query, 
    n_tasks=args.n_validation_tasks
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=args.n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

# Model

In [7]:
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier, TIM, RelationNetworks

class CNN(nn.Module) :
    def __init__(self, name='efficientnetv2_rw_m', num_classes=0)  :
        super(CNN, self).__init__()
        self.model = timm.create_model(name, pretrained=True, num_classes=num_classes)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten(1)
        
        self.use_fc = True
        
    def forward(self, x) :

        if self.use_fc:
            x = self.model(x)

        else :
            x = self.model.forward_features(x)
            x = self.pool(x)
            x = self.flatten(x)

        return x
    
    def set_use_fc(self, x) :
        self.use_fc = x
        
model = CNN(num_classes=len(set(train_set.get_labels()))).to(args.DEVICE)
few_shot_classifier = PrototypicalNetworks(model).to(args.DEVICE)

# Train

In [8]:
from torch.optim import SGD, Optimizer, AdamW, Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


criterion = nn.CrossEntropyLoss()

n_epochs = 20
scheduler_milestones = [3, 6]
scheduler_gamma = 0.1
learning_rate = 1e-3
tb_logs_dir = './tensorboard/FSL_Classical_album_effi2_m'

train_optimizer = Adam(
    few_shot_classifier.parameters(), lr=learning_rate, weight_decay=5e-4
)


train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=tb_logs_dir)


In [9]:
def training_epoch(model_: nn.Module, data_loader: DataLoader, optimizer: Optimizer):
    all_loss = []
    model_.train()
    with tqdm(data_loader, total=len(data_loader), desc="Training") as tqdm_train:
        for images, labels in tqdm_train:
            optimizer.zero_grad()

            loss = criterion(model_(images.to(args.DEVICE)), labels.to(args.DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [ ]:
from easyfsl.methods.utils import evaluate


# best_state = model.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10

for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(model, train_loader, train_optimizer)

#     if epoch % validation_frequency == validation_frequency - 1:

        # We use this very convenient method from EasyFSL's ResNet to specify
        # that the model shouldn't use its last fully connected layer during validation.
    model.set_use_fc(False)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device=args.DEVICE, tqdm_prefix="Validation"
    )
    model.set_use_fc(True)

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = model.state_dict()
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    tb_writer.add_scalar("Train/loss", average_loss, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Validation: 100%|█████████████████████████████████████████████████████| 200/200 [00:32<00:00,  6.16it/s, accuracy=0.63]


Ding ding ding! We found a new best model!
Epoch 1


Validation: 100%|████████████████████████████████████████████████████| 200/200 [00:32<00:00,  6.19it/s, accuracy=0.633]


Ding ding ding! We found a new best model!
Epoch 2


...

Validation: 100%|████████████████████████████████████████████████████| 200/200 [00:32<00:00,  6.12it/s, accuracy=0.667]


Epoch 6


Validation: 100%|████████████████████████████████████████████████████| 200/200 [00:33<00:00,  6.02it/s, accuracy=0.665]


Epoch 7


Training:  54%|███████████████████████████████                           | 241/450 [02:09<01:41,  2.05it/s, loss=0.957]